In [ ]:
import pandas as pd
!pip install alpha_vantage autogluon
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.sectorperformance import SectorPerformances
import matplotlib.pyplot as plt
import requests
import os
from time import sleep
from autogluon.tabular import TabularDataset, TabularPredictor



In [163]:
API_KEY = os.getenv('API_KEY')
symbol = 'BMW.FRK'
pcent_change_to_buy_on = 1.5
sleep_time = 14

In [164]:
# # get sector overview for stock
# url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}'
# r = requests.get(url)
# sector = r.json()['Sector']
# data = r.json()
# print(sector)
# print(data)

In [165]:
# # ***************Sentiment would be a good source but can't seem to get more than 50 items from the API so won't go back
# # far enough???????
# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# # url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={API_KEY}
# url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&time_from=20040825T0130&apikey={API_KEY}'
# print(url)
# r = requests.get(url)

# print(data)

In [166]:
# Get json object with the intraday data and another with  the call's metadata
# For the default date string index behavior
# ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')
# ts_data, ts_meta_data = ts.get_intraday(symbol, interval='1min', outputsize='full')


# ts_data['4. close'].plot()
# plt.title(f'Intraday Times Series for the {symbol} stock (1 min)')
# plt.show()

In [167]:
# Adjusted daily data
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')
ts_data, ts_meta_data = ts.get_daily_adjusted(symbol, outputsize='full')


# ts_data['4. close'].plot()
# plt.title('Intraday Times Series for the GOOGL stock (1 min)')
# plt.show()

In [168]:
adjusted_data = ts_data
adjusted_data.tail()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
index,,,,,,,,,
4562,2005-01-07,34.60,34.72,34.30,34.57,21.4574,280363.0,0.0,1.0
4563,2005-01-06,34.55,34.90,34.53,34.70,21.5381,250466.0,0.0,1.0
4564,2005-01-05,34.20,34.70,34.15,34.65,21.5071,734946.0,0.0,1.0
4565,2005-01-04,33.70,34.48,33.67,34.39,21.3457,567720.0,0.0,1.0
4566,2005-01-03,33.30,33.80,33.30,33.75,20.9485,246820.0,0.0,1.0


In [169]:
# compute the percent change for each day
adjusted_data['day_pcent_change'] = 100 * ((ts_data['4. close'] - ts_data['1. open']) / ts_data['1. open'])


# ******need to predict whether price for tomorrow will go up by 2% or greater - so need label - buy - 0 or 1
# today's close data will be used to predict whether to buy tomorrow
# so model cannot have the most recent row in it, since that would be today's data
# from which tomorrow's prediction will be made

# shift all the next days data by one which will be used to determine whether it should have been bought or not
adjusted_data['next_days_pcent_change'] = adjusted_data['day_pcent_change'].shift(+1)

In [170]:
# create the label - binary classification of whether to buy or not based on whether the stock went up 2% or more
# that day

def buy_or_not(row):
    if row.next_days_pcent_change >= pcent_change_to_buy_on:
        return 1
    else:
        return 0

adjusted_data['buy_tomorrow'] = adjusted_data.apply(buy_or_not, axis='columns')

# next_days_pcent_change will pollute the data set so drop it

adjusted_data.drop(columns='next_days_pcent_change',
                  inplace=True)

In [171]:
# technical indicators - some of these are premium

wma = [5, 10, 20, 40, 80]

# def concat_wma_df(df, period):
#     '''
#     concatenate the pandas dataframes being passed into the function
#     '''
    
wma_data = pd.DataFrame()

for period in wma:
    
    sleep(sleep_time) # avoid breaching the API request rate limit
    ti = TechIndicators(key=API_KEY, output_format='pandas')
    ti_data, ti_meta_data = ti.get_wma(symbol, interval='daily', time_period=period)
    wma_data[f'WMA_{period}'] = ti_data['WMA']

    
wma_data.describe() 

# *********get the resistance figures for the MAs?
# need to understand the relevance of the MAs better and what predictive power they are supposed to be happened so it can be represented properly




# ti_data.plot()
# plt.title('WMA indicator for  GOOGL stock (60 min)')
# plt.show()

,WMA_5,WMA_10,WMA_20,WMA_40,WMA_80
count,4563.000000,4558.000000,4548.000000,4528.000000,4488.000000
mean,51.404965,51.415227,51.436436,51.479705,51.573188
std,22.548574,22.523558,22.473208,22.370837,22.182938
min,12.127500,12.280200,12.566200,13.270200,13.574300
25%,27.754950,27.830800,27.801750,27.826525,27.970875
50%,55.784000,56.123450,56.754950,56.965300,56.475250
75%,70.571500,70.595975,70.558800,70.575300,70.554200
max,98.884000,98.297600,96.555600,94.075400,92.770000


In [172]:
# tech indicator data is received in the reverse order to price data so reverse it
rev_wma_data = wma_data[::-1]
rev_wma_data.reset_index(inplace=True)

# rev_ti_data.drop(index=rev_ti_data.index[0], 
#         axis=0, 
#         inplace=True)
rev_wma_data.head()

,date,WMA_5,WMA_10,WMA_20,WMA_40,WMA_80
0,2022-12-23,83.6487,83.5449,83.8986,83.8549,81.3006
1,2022-12-22,83.5940,83.5547,83.9561,83.8272,81.1762
2,2022-12-21,83.6267,83.6218,84.0520,83.8128,81.0576
3,2022-12-20,83.3787,83.5673,84.0758,83.7557,80.9173
4,2022-12-19,83.2560,83.5925,84.1285,83.7050,80.7809


In [173]:
for period in wma:
    adjusted_data[f'WMA_{period}'] = rev_wma_data[f'WMA_{period}']
#     these should be percentage diff to close as well?
    adjusted_data[f'WMA_{period}_pcent_diff_to_close'] = (adjusted_data['4. close'] - adjusted_data[f'WMA_{period}']) / adjusted_data['4. close']
    adjusted_data[f'WMA_{period}_pcent_diff_to_open'] = (adjusted_data['1. open'] - adjusted_data[f'WMA_{period}']) / adjusted_data['4. close']
    adjusted_data.drop(columns=f'WMA_{period}', inplace=True)

In [174]:
adjusted_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,WMA_5_pcent_diff_to_close,WMA_5_pcent_diff_to_open,WMA_10_pcent_diff_to_close,WMA_10_pcent_diff_to_open,WMA_20_pcent_diff_to_close,WMA_20_pcent_diff_to_open,WMA_40_pcent_diff_to_close,WMA_40_pcent_diff_to_open,WMA_80_pcent_diff_to_close,WMA_80_pcent_diff_to_open
index,,,,,,,,,,,,,,,,,,,,,
0,2022-12-23,83.41,83.72,83.24,83.65,83.65,431.0,0.0,1.0,0.287735,...,0.000016,-0.002854,0.001256,-0.001613,-0.002972,-0.005841,-0.002449,-0.005319,0.028086,0.025217
1,2022-12-22,84.20,84.20,83.01,83.29,83.29,443.0,0.0,1.0,-1.080760,...,-0.003650,0.007276,-0.003178,0.007748,-0.007997,0.002928,-0.006450,0.004476,0.025379,0.036304
2,2022-12-21,83.21,84.20,83.21,84.04,84.04,60.0,0.0,1.0,0.997476,...,0.004918,-0.004958,0.004976,-0.004900,-0.000143,-0.010019,0.002703,-0.007173,0.035488,0.025612
3,2022-12-20,82.17,83.75,82.17,83.75,83.75,742.0,0.0,1.0,1.922843,...,0.004433,-0.014432,0.002181,-0.016684,-0.003890,-0.022756,-0.000068,-0.018934,0.033823,0.014958
4,2022-12-19,82.66,83.86,82.66,83.70,83.70,281.0,0.0,1.0,1.258166,...,0.005305,-0.007121,0.001284,-0.011141,-0.005119,-0.017545,-0.000060,-0.012485,0.034876,0.022450


In [175]:
# ****************
# Now get bollinger band details - need to study this more and figure out what it means
# https://www.tradingview.com/support/solutions/43000501972-bollinger-bands-width-bbw/

# calculate the bollinger band width then look at it the difference over the last 6 months
# Bollinger Bands Width = (Upper Band - Lower Band) / Middle Band

# look for: The low volatility period is followed by a surge in volatility and 
# price breaks through the Upper Band or falls through the Lower Band signifying a 
# change in the sideways movement and the beginning of a new directional trend.


# In a Bullish BBW Squeeze

# BBW drops. (In the example below, the threshold is 6% however this changes from security to security and 
#             timeframe to timeframe)
# Price breaks through the Upper Band which starts a new upward trend. Volatility also increases.


# In a Bearish BBW Squeeze

# BBW drops. (In the example below, the threshold is 9% however this changes from security to 
#             security and timeframe to timeframe).
# Price falls below the Lower Band which starts a new downward trend. Volatility also increases.
        
bb = TechIndicators(key=API_KEY, output_format='pandas')
bb_data, bb_meta_data = bb.get_bbands(symbol, interval='daily', time_period=20)

In [176]:
bb_data['bb_width'] = (bb_data['Real Upper Band'] - bb_data['Real Lower Band']) / bb_data['Real Middle Band']
bb_data.reset_index(inplace=True)
bb_data.head()

,date,Real Upper Band,Real Middle Band,Real Lower Band,bb_width
0,2022-12-23,86.2285,84.2410,82.2535,0.047186
1,2022-12-22,86.2293,84.2540,82.2787,0.046889
2,2022-12-21,86.2236,84.2975,82.3714,0.045698
3,2022-12-20,86.2211,84.2890,82.3569,0.045845
4,2022-12-19,86.2235,84.3035,82.3835,0.045550


In [177]:
adjusted_data['bb_width'] = bb_data['bb_width']
adjusted_data['bb_upper'] = bb_data['Real Upper Band']
adjusted_data['bb_lower'] = bb_data['Real Lower Band']
# add the below to percentages like day_pcent_change?
adjusted_data['pcent_diff_to_bb_upper'] = (adjusted_data['4. close'] - adjusted_data['bb_upper']) / adjusted_data['4. close']
adjusted_data['pcent_diff_to_bb_lower'] = (adjusted_data['4. close'] - adjusted_data['bb_lower']) / adjusted_data['4. close']

In [178]:
# # Don't seem to be able to get this with sufficient historical data
# sp = SectorPerformances(key=API_KEY, output_format='pandas')
# sp_data, sp_meta_data = sp.get_sector()
# sp_data.head()
# # sp_data['Rank A: Real-Time Performance'].plot(kind='bar')
# # plt.title('Real Time Performance (%) per Sector')
# # plt.tight_layout()
# # plt.grid()
# # plt.show()

In [179]:
macd = TechIndicators(key=API_KEY, output_format='pandas')
sleep(sleep_time)
macd_data, macd_meta_data = bb.get_macd(symbol, series_type='close', interval='daily')
macd_data.reset_index(inplace=True)
adjusted_data['macd_signal'] = macd_data['MACD_Signal']

In [180]:
stochrsi = TechIndicators(key=API_KEY, output_format='pandas')
sleep(sleep_time)
stochrsi_data, stochrsi_meta_data = bb.get_stochrsi(symbol, series_type='close', interval='daily', time_period=10, fastkperiod=6, fastdmatype=1)
stochrsi_data.reset_index(inplace=True)
adjusted_data['stochrsi_fast_k'] = stochrsi_data['FastK']
adjusted_data['stochrsi_fast_d'] = stochrsi_data['FastD']

In [181]:
ppo = TechIndicators(key=API_KEY, output_format='pandas')
sleep(sleep_time)
ppo_data, ppo_meta_data = bb.get_ppo(symbol, series_type='close', interval='daily', fastperiod=10, matype=1)
ppo_data.reset_index(inplace=True)
adjusted_data['ppo'] = ppo_data['PPO']

In [182]:
adjusted_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
index,,,,,,,,,,,,,,,,,,,,,
0,2022-12-23,83.41,83.72,83.24,83.65,83.65,431.0,0.0,1.0,0.287735,...,0.025217,0.047186,86.2285,82.2535,-0.030825,0.016695,0.6142,72.6758,67.7966,-1.4906
1,2022-12-22,84.20,84.20,83.01,83.29,83.29,443.0,0.0,1.0,-1.080760,...,0.036304,0.046889,86.2293,82.2787,-0.035290,0.012142,0.6855,44.1339,62.9174,-1.2474
2,2022-12-21,83.21,84.20,83.21,84.04,84.04,60.0,0.0,1.0,0.997476,...,0.025612,0.045698,86.2236,82.3714,-0.025983,0.019855,0.7657,100.0000,81.7008,-1.1117
3,2022-12-20,82.17,83.75,82.17,83.75,83.75,742.0,0.0,1.0,1.922843,...,0.014958,0.045845,86.2211,82.3569,-0.029506,0.016634,0.8469,88.6782,63.4016,-0.8960
4,2022-12-19,82.66,83.86,82.66,83.70,83.70,281.0,0.0,1.0,1.258166,...,0.022450,0.045550,86.2235,82.3835,-0.030149,0.015729,0.9447,67.6404,38.1249,-0.7531


In [183]:
# clean up NaNs
final_data = adjusted_data.dropna().reset_index(drop=True)


In [184]:
final_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
0,2022-12-23,83.41,83.72,83.24,83.65,83.65,431.0,0.0,1.0,0.287735,...,0.025217,0.047186,86.2285,82.2535,-0.030825,0.016695,0.6142,72.6758,67.7966,-1.4906
1,2022-12-22,84.20,84.20,83.01,83.29,83.29,443.0,0.0,1.0,-1.080760,...,0.036304,0.046889,86.2293,82.2787,-0.035290,0.012142,0.6855,44.1339,62.9174,-1.2474
2,2022-12-21,83.21,84.20,83.21,84.04,84.04,60.0,0.0,1.0,0.997476,...,0.025612,0.045698,86.2236,82.3714,-0.025983,0.019855,0.7657,100.0000,81.7008,-1.1117
3,2022-12-20,82.17,83.75,82.17,83.75,83.75,742.0,0.0,1.0,1.922843,...,0.014958,0.045845,86.2211,82.3569,-0.029506,0.016634,0.8469,88.6782,63.4016,-0.8960
4,2022-12-19,82.66,83.86,82.66,83.70,83.70,281.0,0.0,1.0,1.258166,...,0.022450,0.045550,86.2235,82.3835,-0.030149,0.015729,0.9447,67.6404,38.1249,-0.7531


In [185]:
# don't need the dates
final_data.drop(columns='index', inplace=True)


In [186]:
final_data = final_data.iloc[1: , :]
final_data.head(60)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,buy_tomorrow,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
1,84.20,84.20,83.01,83.29,83.29,443.0,0.0,1.0,-1.080760,0,...,0.036304,0.046889,86.2293,82.2787,-0.035290,0.012142,0.6855,44.1339,62.9174,-1.2474
2,83.21,84.20,83.21,84.04,84.04,60.0,0.0,1.0,0.997476,0,...,0.025612,0.045698,86.2236,82.3714,-0.025983,0.019855,0.7657,100.0000,81.7008,-1.1117
3,82.17,83.75,82.17,83.75,83.75,742.0,0.0,1.0,1.922843,0,...,0.014958,0.045845,86.2211,82.3569,-0.029506,0.016634,0.8469,88.6782,63.4016,-0.8960
4,82.66,83.86,82.66,83.70,83.70,281.0,0.0,1.0,1.258166,1,...,0.022450,0.045550,86.2235,82.3835,-0.030149,0.015729,0.9447,67.6404,38.1249,-0.7531
5,83.50,83.50,82.33,82.65,82.65,657.0,0.0,1.0,-1.017964,0,...,0.034571,0.045899,86.2280,82.3590,-0.043291,0.003521,1.0563,0.0000,8.6094,-0.7096
6,83.86,83.86,82.80,82.80,82.80,695.0,0.0,1.0,-1.264011,0,...,0.040249,0.042389,86.1505,82.5745,-0.040465,0.002723,1.1831,0.0000,17.2188,-0.6703
7,83.92,84.31,83.53,83.58,83.58,907.0,0.0,1.0,-0.405148,0,...,0.042046,0.044880,86.2285,82.4435,-0.031688,0.013598,1.3005,28.5771,34.4376,-0.7425
8,84.96,85.00,83.98,84.18,84.18,1165.0,0.0,1.0,-0.918079,0,...,0.055808,0.047423,86.2907,82.2933,-0.025074,0.022413,1.4011,46.3193,40.2982,-0.7287
9,83.95,84.54,83.95,84.54,84.54,733.0,0.0,1.0,0.702799,0,...,0.045531,0.047432,86.3059,82.3071,-0.020888,0.026412,1.4911,43.0797,34.2772,-0.7640
10,82.95,84.62,82.95,84.51,84.51,219.0,0.0,1.0,1.880651,0,...,0.035771,0.047546,86.3189,82.3101,-0.021405,0.026031,1.5772,42.3920,25.4746,-0.7512


In [187]:
final_data.describe()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,buy_tomorrow,...,WMA_80_pcent_diff_to_open,bb_width,bb_upper,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,stochrsi_fast_k,stochrsi_fast_d,ppo
count,4487.000000,4487.000000,4487.000000,4487.000000,4487.000000,4.487000e+03,4487.000000,4487.0,4487.000000,4487.000000,...,4487.000000,4487.000000,4487.000000,4487.000000,4487.000000,4487.000000,4487.000000,4487.000000,4487.000000,4487.000000
mean,63.986331,64.655229,63.251593,63.988046,51.927019,1.067083e+05,0.007114,1.0,0.025061,0.154446,...,0.217600,0.119015,54.706068,48.881770,0.169230,0.262392,0.095825,51.097660,51.096564,0.188645
std,21.954073,22.092552,21.767080,21.940089,22.396761,2.338250e+05,0.139861,0.0,1.726769,0.361416,...,0.139273,0.070676,23.384900,21.393248,0.143250,0.127161,0.996802,40.766556,31.355546,2.388457
min,17.500000,17.905000,16.115000,17.400000,11.657600,0.000000e+00,0.000000,1.0,-12.293578,0.000000,...,-0.538792,0.025871,14.511300,10.639000,-0.734486,-0.054526,-5.475400,0.000000,0.110700,-15.330200
25%,43.400000,43.815000,42.835000,43.405000,28.243850,2.059300e+04,0.000000,1.0,-0.882058,0.000000,...,0.109805,0.071784,29.500400,26.781300,0.062726,0.157322,-0.393000,2.335900,21.717950,-1.113700
50%,68.570000,69.320000,67.830000,68.810000,57.046900,3.915700e+04,0.000000,1.0,0.040239,0.000000,...,0.250330,0.102939,61.092300,53.001800,0.204304,0.292289,0.112600,52.158900,51.212000,0.244700
75%,82.255000,83.130000,81.400000,82.236500,70.821300,9.639000e+04,0.000000,1.0,0.929095,0.000000,...,0.335002,0.142794,73.649050,67.665650,0.286337,0.370367,0.616950,100.000000,80.599450,1.697450
max,122.890000,123.699000,120.225000,122.998000,99.650000,5.277522e+06,4.000000,1.0,16.092245,1.000000,...,0.507412,0.662782,102.715400,91.049500,0.382821,0.547382,3.382800,100.000000,99.996200,6.647600


In [188]:
train_data = final_data
label = 'buy_tomorrow'
save_path = './autogluon_models'

In [189]:
# there is currently a bug where num_bag_sets >1 produces an error with various algos - waiting on fix
# https://github.com/autogluon/autogluon/issues/2581

predictor = TabularPredictor(label=label, path=save_path, verbosity=3, eval_metric='precision').fit(
    train_data, presets='best_quality', time_limit=15000, num_bag_folds=20, num_stack_levels=3
)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'num_bag_folds': 20, 'num_stack_levels': 3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 20,
 'num_bag_sets': None,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=20, num_bag_sets=20
Saving ./autogluon_models/learner.pkl
Saving ./autogluon_models/predictor.pkl
Beginning AutoGluon training ... Time lim

In [190]:
results = predictor.fit_summary()

Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBM_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/CatBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./autogluon_models/models/XGBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./autogluon

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           CatBoost_BAG_L1   1.000000       0.028020  19.758722                0.028020          19.758722            1       True          7
1           LightGBM_BAG_L2   0.758621       2.482328  24.534868                0.106126          19.512355            2       True         16
2      LightGBMLarge_BAG_L2   0.687500       2.515875  49.400836                0.139673          44.378322            2       True         25
3            XGBoost_BAG_L4   0.681818       4.935526  44.152152                0.387177          28.890719            4       True         47
4         LightGBMXT_BAG_L2   0.655172       2.496622  22.081645                0.120420          17.059132            2       True         15
5         LightGBMXT_BAG_L4   0.655172       4.682258  31.960919                

In [191]:
predictor.leaderboard(train_data, silent=True)

Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBM_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/CatBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./autogluon_models/models/XGBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./autogluon

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,1.000000,1.000000,0.038095,0.028020,19.758722,0.038095,0.028020,19.758722,1,True,7
1,LightGBM_BAG_L1,1.000000,0.557692,0.214824,0.092479,17.649890,0.214824,0.092479,17.649890,1,True,4
2,RandomForestEntr_BAG_L1,1.000000,0.208791,0.224058,0.256319,2.100045,0.224058,0.256319,2.100045,1,True,6
3,RandomForestGini_BAG_L1,1.000000,0.224490,0.225853,0.266957,1.301742,0.225853,0.266957,1.301742,1,True,5
4,ExtraTreesGini_BAG_L1,1.000000,0.180851,0.235512,0.266096,0.905742,0.235512,0.266096,0.905742,1,True,8
5,ExtraTreesEntr_BAG_L1,1.000000,0.202381,0.236475,0.269624,0.697529,0.236475,0.269624,0.697529,1,True,9
6,LightGBMXT_BAG_L1,1.000000,0.465116,0.304072,0.110265,15.611301,0.304072,0.110265,15.611301,1,True,3
7,XGBoost_BAG_L1,1.000000,0.476923,0.350253,0.389956,31.946066,0.350253,0.389956,31.946066,1,True,11
8,LightGBMLarge_BAG_L1,1.000000,0.491228,0.517237,0.161006,36.715800,0.517237,0.161006,36.715800,1,True,13
9,KNeighborsDist_BAG_L1,1.000000,0.122807,0.605458,0.607851,0.008561,0.605458,0.607851,0.008561,1,True,2


In [192]:
predictor.feature_importance(train_data)

These features in provided data are not utilized by the predictor and will be ignored: ['8. split coefficient']
Loading: ./autogluon_models/models/WeightedEnsemble_L3/model.pkl
Computing feature importance via permutation shuffling for 27 features using 4487 rows with 5 shuffle sets...
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L2/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L2/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L3/model.pkl
	367.04s	= Expected runtime (73.41s per shuffle set)
Loading: ./autogluon_models/models/ExtraTreesEntr_B

,importance,stddev,p_value,n,p99_high,p99_low
7. dividend amount,0.000000,0.000000,0.500000,5,0.000000,0.000000
day_pcent_change,-0.003655,0.039270,0.577358,5,0.077203,-0.084514
ppo,-0.021273,0.043738,0.831031,5,0.068784,-0.111329
bb_width,-0.071032,0.078455,0.943542,5,0.090507,-0.232571
WMA_20_pcent_diff_to_close,-0.094851,0.030117,0.998928,5,-0.032839,-0.156863
WMA_5_pcent_diff_to_close,-0.103003,0.028470,0.999366,5,-0.044384,-0.161623
stochrsi_fast_k,-0.106143,0.015855,0.999942,5,-0.073498,-0.138789
stochrsi_fast_d,-0.119357,0.058997,0.994686,5,0.002118,-0.240833
WMA_20_pcent_diff_to_open,-0.120010,0.039272,0.998800,5,-0.039148,-0.200871
WMA_10_pcent_diff_to_open,-0.125198,0.022575,0.999878,5,-0.078717,-0.171679
